In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

# Aquisição dos dados

In [3]:
test_set = pd.read_csv(r'./eel891-202101-trabalho-1/conjunto_de_teste.csv')
test_set

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,estado_onde_trabalha,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha
0,20001,1,25,presencial,1,M,37,2,0,0,...,,N,,0,0.0,0.0,0.0,0.0,384.0,384.0
1,20002,1,10,internet,1,F,31,2,0,0,...,RJ,N,,0,9.0,5.0,NaN,NaN,275.0,275.0
2,20003,1,10,internet,1,F,18,2,0,0,...,RS,N,,0,9.0,2.0,NaN,NaN,948.0,948.0
3,20004,1,10,presencial,1,F,55,2,0,0,...,,N,,0,9.0,1.0,0.0,0.0,581.0,581.0
4,20005,1,10,presencial,1,F,55,1,0,0,...,,N,,0,0.0,1.0,0.0,0.0,573.0,573.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,24996,1,10,presencial,1,F,49,2,1,0,...,,N,,0,9.0,1.0,0.0,0.0,652.0,652.0
4996,24997,1,5,internet,1,M,18,2,0,0,...,,N,,0,NaN,NaN,NaN,NaN,855.0,855.0
4997,24998,1,10,presencial,1,F,29,1,0,0,...,,N,,0,9.0,5.0,NaN,NaN,371.0,371.0
4998,24999,1,25,internet,1,F,31,2,2,0,...,,N,,0,NaN,NaN,NaN,NaN,670.0,670.0


In [4]:
train_set = pd.read_csv(r'./eel891-202101-trabalho-1/conjunto_de_treinamento.csv')
train_set

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
0,1,1,10,presencial,1,M,85,2,0,0,...,N,,0,9.0,1.0,0.0,0.0,600.0,600.0,0
1,2,1,25,internet,1,F,38,1,0,0,...,N,,0,2.0,5.0,NaN,NaN,492.0,492.0,0
2,3,1,20,internet,1,F,37,2,0,0,...,N,,0,NaN,NaN,NaN,NaN,450.0,450.0,1
3,4,1,20,internet,1,M,37,1,1,0,...,Y,54,0,9.0,2.0,NaN,NaN,932.0,932.0,1
4,5,7,1,internet,1,F,51,1,3,0,...,N,,0,9.0,5.0,NaN,NaN,440.0,440.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,1,10,presencial,1,M,27,2,0,0,...,N,,0,9.0,1.0,0.0,0.0,308.0,308.0,0
19996,19997,1,20,presencial,1,F,26,2,1,0,...,N,,0,9.0,2.0,0.0,0.0,639.0,639.0,0
19997,19998,1,10,internet,1,F,63,2,0,0,...,N,,0,9.0,1.0,NaN,NaN,486.0,486.0,0
19998,19999,1,5,internet,1,F,84,1,0,0,...,N,,0,NaN,NaN,NaN,NaN,590.0,590.0,0


training = True

if training:
    df = train_set
else:
    df = test_set

In [ ]:
def get_metrics(y_teste_a, y_resposta_a):

    df_result = pd.DataFrame(
        {
            'teste': y_teste_a,
            'resposta': y_resposta_a
        }
    )
    df_result['type'] = np.select(
        [
            (df_result['resposta'] == 1) & (df_result['teste'] == 1),
            (df_result['resposta'] == 1) & (df_result['teste'] == 0),
            (df_result['resposta'] == 0) & (df_result['teste'] == 0),
            (df_result['resposta'] == 0) & (df_result['teste'] == 1)
        ],
        [
            'TP',
            'FN',
            'TN',
            'FP',
        ]
    )

    dict_analise = df_result['type'].value_counts().to_dict()
    
    return {
        'Acuracia': (dict_analise['TP']+dict_analise['TN'])/(dict_analise['TP']+dict_analise['FP']+dict_analise['TN']+dict_analise['FN']),
        'Sensibilidade': dict_analise['TP']/(dict_analise['TP']+dict_analise['FN']),
        'Especificidade': dict_analise['TN']/(dict_analise['TN']+dict_analise['FP'])    
    }

df_credito = df_credito.replace('', np.nan)
df_credito = df_credito.replace(' ', np.nan)

columns_to_exclude = [
    'id_solicitante',
    'estado_onde_trabalha',
    'codigo_area_telefone_trabalho',
    'profissao_companheiro',
    'grau_instrucao_companheiro',
    'possui_telefone_celular',
    'meses_no_trabalho',
    'codigo_area_telefone_trabalho',
    'codigo_area_telefone_residencial',
    'grau_instrucao'
]
df_credito = df_credito.drop(columns=columns_to_exclude)
df_credito['renda_mensal_regular']

# Categorico

In [ ]:
df_credito.columns[df_credito.dtypes == object]

## Categorico Nominal

In [ ]:
colunas_categorias_nominais = [
    'produto_solicitado', 'dia_vencimento', 'forma_envio_solicitacao', 'tipo_endereco', 'sexo', 'estado_civil', 'nacionalidade', 'estado_onde_nasceu',
    'estado_onde_reside', 'possui_telefone_residencial', 'tipo_residencia', 'vinculo_formal_com_empresa',
    'possui_telefone_trabalho','possui_cartao_diners', 'possui_cartao_amex', 'possui_outros_cartoes','possui_email',
    'possui_cartao_visa','possui_cartao_mastercard', 'possui_carro', 'possui_telefone_trabalho', 'vinculo_formal_com_empresa',
    'profissao', 'ocupacao', 'local_onde_reside', 'local_onde_trabalha'
]

### Analisando a cardinalidade com intuito de remover alguns campos com alta cardinalidade.

In [ ]:
df_credito[colunas_categorias_nominais].nunique().sort_values()

## Binarios

In [ ]:
colunas_binarias = df_credito[colunas_categorias_nominais].nunique()[df_credito[colunas_categorias_nominais].nunique() == 2]
colunas_binarias
binarizador = LabelBinarizer()
for v in colunas_binarias.index:
    df_credito[v] = binarizador.fit_transform(df_credito[v])
colunas_binarias.index
colunas_categorias_nominais_restantes = [x for x in colunas_categorias_nominais if x not in colunas_binarias.index]

In [ ]:
colunas_categorias_nominais_para_excluir = ['estado_onde_nasceu', 'local_onde_reside', 'local_onde_trabalha']
df_credito = df_credito.drop(columns=colunas_categorias_nominais_para_excluir)
colunas_categorias_nominais_restantes = [x for x in colunas_categorias_nominais_restantes if x not in colunas_categorias_nominais_para_excluir]